## Load data and save the Document object

In [1]:
from pathlib import Path
from src.utils import  final_query, fulldocpath, fulllnodespath
import asyncio
import nest_asyncio
nest_asyncio.apply()
# fulldocpath = Path.joinpath(docs_path, "10K_esg")
# fulllnodespath = Path.joinpath(nodes_path)
sq_vec_summary_engine = final_query(fulldocpath, fulllnodespath)

The path /Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/llamadocs/10K_esg exists. Loading from disk...
The path /Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/nodes/tesla_esg_nodes.pkl exists. Loading from disk...
loading index 10K_esg


In [16]:
# esg_query = "How much was Tesla's Research and Development amount and as a percentage of revenue in 2020?"
# esg_query = "How much did McKinsey invested in strengthening risk management since 2018?"
# esg_query = "How much was Tesla's Selling, General, and Administrative expense in 2022?"
# esg_query = "How much was Tesla's Selling General and Administrative expense in 2022 as a percentage of revenue?"
esg_query = "How much did Tesla reported as changes in Restructuring and Other in 2022 vs 2021? "
# esg_query = "How much did Tesla's Selling, General, and Administrative changed from 2022 vs 2021 in dollar amount? "
response = sq_vec_summary_engine.query(esg_query)
print(response)


Generated 2 sub questions.
[Tesla10K_Esg_Report_Query_Engine] Q: What was the value of Restructuring and Other in Tesla's 2022 10K report?
[Tesla10K_Esg_Report_Query_Engine] Q: What was the value of Restructuring and Other in Tesla's 2021 10K report?
[Tesla10K_Esg_Report_Query_Engine] A: According to the table provided in the context, the value of Restructuring and Other in Tesla's 2021 10K report was ($27).
[Tesla10K_Esg_Report_Query_Engine] A: According to the table provided in the context, the value of Restructuring and Other charges for Tesla in 2022 was $176 million.
Tesla reported an increase of $203 million in Restructuring and Other charges from 2021 to 2022. In 2021, the value was ($27) million, while in 2022 it was $176 million, representing a year-over-year increase of $203 million.


In [3]:
# sales_query ="What is the total sales in Los Angeles"
# sales_query ="Which Segment sold the most?" 
# response = sales_sql_query_engine.query(sales_query)
# print(response)

In [8]:
# sales_query ="What is the total sales in Los Angeles"
# sales_query ="Which product Segment sold the most using the sales database?"
sales_query ="How much did Tesla's Selling, General, and Administrative changed from 2022 vs 2021 in dollar amount? "
# sales_query = "How much was Tesla's Selling General and Administrative expense in 2021 as a percentage of revenues?"
response = sq_vec_summary_engine.query(sales_query)
print(str(response))

Generated 3 sub questions.
[Tesla10K_Esg_Report_Query_Engine] Q: What was Tesla's Selling, General, and Administrative expenses in 2022?
[Tesla10K_Esg_Report_Query_Engine] Q: What was Tesla's Selling, General, and Administrative expenses in 2021?
[Tesla10K_Esg_Report_Query_Engine] Q: What is the difference in Tesla's Selling, General, and Administrative expenses between 2022 and 2021?
[Tesla10K_Esg_Report_Query_Engine] A: Tesla's Selling, General, and Administrative expenses in 2022 were $3,946 million.
[Tesla10K_Esg_Report_Query_Engine] A: Tesla's Selling, General, and Administrative expenses in 2021 were $4,517 million.
[Tesla10K_Esg_Report_Query_Engine] A: The difference in Tesla's Selling, General, and Administrative (SG&A) expenses between 2022 and 2021 was a decrease of $571 million, or 13%. Specifically, the SG&A expenses were $3,946 million in 2022, compared to $4,517 million in 2021, representing a decrease of $571 million or 13% year-over-year.
The difference in Tesla's Selli

In [9]:
# esg_query = "How is McKinsey responding to the war in Ukraine in its sustainability effort?"
# esg_query ="How does Deloitte's sustainbaility goals differs from that of McKinsey"
# esg_query = "Summarize Deloitte ESg report"
# esg_query = "How much was Tesla's Selling, General, and Administrative expense in 2022?"
# esg_query = "How much did Tesla's Selling, General, and Administrative changed from 2022 vs 2021 in dollar amount? "
esg_query = "How much was Tesla's Selling General and Administrative  in 2022 vs 2021 as a percentage of revenues?"
response = sq_vec_summary_engine.query(esg_query)
print(response)

Generated 4 sub questions.
[Tesla10K_Esg_Report_Query_Engine] Q: What was Tesla's Selling General and Administrative expenses in 2022?
[Tesla10K_Esg_Report_Query_Engine] Q: What was Tesla's Selling General and Administrative expenses in 2021?
[Tesla10K_Esg_Report_Query_Engine] Q: What were Tesla's total revenues in 2022?
[Tesla10K_Esg_Report_Query_Engine] Q: What were Tesla's total revenues in 2021?
[Tesla10K_Esg_Report_Query_Engine] A: Tesla's Selling, General and Administrative expenses in 2021 were $4,517 million.
[Tesla10K_Esg_Report_Query_Engine] A: Tesla's total revenues in 2022 were $81,462 million.
[Tesla10K_Esg_Report_Query_Engine] A: Tesla's total revenues in 2021 were $53,823 million.
[Tesla10K_Esg_Report_Query_Engine] A: Tesla's Selling, General and Administrative (SG&A) expenses in 2022 were $3,946 million.
In 2022, Tesla's Selling, General and Administrative (SG&A) expenses were $3,946 million, which represented 4.8% of its total revenues of $81,462 million for that year.